# Transforming SFHAs

This notebook experiments with transforming data from REST API queries in-place, either in pandas dataframes or python dicts.

## Extracting corrected LOMRs and all flood delineations inside those LOMRs

In [ ]:
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureSet, FeatureLayer, FeatureLayerCollection, SpatialDataFrame
from arcgis.geometry import Geometry, Point, filters, union, buffer
import pandas as pd

In [ ]:
# Feature services
city_lims_url = "https://maps.bouldercolorado.gov/arcgis/rest/services/plan/CityLimits/MapServer/0"
city = FeatureLayer(city_lims_url)

nfhl_url = "https://hazards.fema.gov/gis/nfhl/rest/services/public/NFHL/MapServer"
nfhl = FeatureLayerCollection(nfhl_url)
lomr = nfhl.layers[1]
sfha = nfhl.layers[27]

In [ ]:
sr = 2876 # NAD83(HARN) / Colorado North (ftUS)

In [ ]:
# Geometry filter object
anon_gis = GIS()
city_lims = city.query(out_sr=sr)
city_geoms = [poly.geometry for poly in city_lims.features]
city_union = union(spatial_ref=sr, geometries=city_geoms, gis=anon_gis)
geom_filter = filters.intersects(city_union, sr=sr)

In [ ]:
# Query
date_str = '2018-08-16'
clause = f"STATUS = 'Effective' AND EFF_DATE >= '{date_str}'"
boulder_lomrs = lomr.query(where=clause,
                           geometry_filter=geom_filter,
                           out_sr=sr)

In [ ]:
# Drop LOMR polygons that have duplicate Case Numbers and Geometries.
temp = boulder_lomrs.sdf
temp['GEOM_STR'] = str(temp['SHAPE'])
temp.drop_duplicates(subset=['CASE_NO', 'GEOM_STR'], inplace=True)
temp.sort_values(by='EFF_DATE', inplace=True, ascending=False)
boulder_lomrs = FeatureSet.from_dataframe(temp)

In [ ]:
# Query BoCo flood areas
flood_areas = sfha.query(where="DFIRM_ID = '08013C'",
                         out_fields=['FLD_AR_ID', 'STUDY_TYP', 'FLD_ZONE', 'ZONE_SUBTY', 'SFHA_TF', 'STATIC_BFE', 'DEPTH'],
                         out_sr=sr)

In [ ]:
# Copy BoCo flood areas and empty the df
flood_sdf = flood_areas.sdf
flood = flood_sdf.copy()
flood.drop(list(range(len(flood))),inplace=True)

In [ ]:
# Loop through LOMRs and flood areas to find polys inside LOMRs
for l in boulder_lomrs.features:
    g = Geometry(l.geometry)
    
    # buffer LOMR geom by one foot to avoid topological
    # errors where polys share an edge
    buf = g.buffer(1)

    for row in flood_areas.features:
        area_id = row.attributes['FLD_AR_ID']
        f = Geometry(row.geometry)
        if buf.contains(f):
            flood = flood.append(flood_sdf[flood_sdf['FLD_AR_ID'] == area_id], ignore_index=True)

# drop any rows that represent duplicate flood areas
flood.drop_duplicates(subset=['FLD_AR_ID'], inplace=True)

## Transform the SFHAs

### Criteria:
- FLOODPLAIN
  - 500-Year = "FLD_ZONE = 'X' AND ZONE_SUBTY IN ('0.2 PCT ANNUAL CHANCE FLOOD HAZARD', 'LEVEE')"
  - 100-Year = "SFHA_TF = 'T'"
    - Conveyance Zone = "SFHA_TF = 'T' AND 'ZONE_SUBTY' = 'FLOODWAY'"
- LIFECYCLE = 'Active'
- FEMAZONE
  - if FLD_ZONE = AO
    - FEMAZONE = FLD_ZONE + str(DEPTH)
  - if FLD_ZONE = AH
    - FEMAZONE = FLD_ZONE + str(STATIC_BFE)
- SOURCE = 'FEMA'
- ADOPTDATE and INEFFDATE (Spatial join of dataframes)
  - If a polygon resides in 2+ different LOMR areas, the join creates 2+ polygons
    - The polygon with the earlier ADOPTDATE gets an INEFFDATE equal to the next later ADOPTDATE
    - The polygon with the latest ADOPTDATE gets a null INEFFDATE
- DRAINAGE
  - Use "set-theoretic" funcs: centroid inside city-floodplain with DRAINAGE = X


In [ ]:
# Calculate ADOPT and INEFFDATEs
def calc_ineffdate(row, date_dict):
    fema_id = row["FLD_AR_ID"]
    adopt_date = row["EFF_DATE"]
    ineff_date = None
    try:
        idx = date_dict[fema_id].index(adopt_date)
        try:
            ineff_date = date_dict[fema_id][idx+1]
        except IndexError:
            # The polygon has the most recent ADOPTDATE of all the LOMRs that touch the polygon
            pass
    except KeyError:
        # The polygon was not duplicated
        pass
    return ineff_date

flood = flood.spatial.join(boulder_lomrs.sdf)
dups = list(flood[flood.duplicated(["FLD_AR_ID"])]["FLD_AR_ID"])
dup_dates = {fld_ar_id: sorted(list(flood[flood["FLD_AR_ID"]==fld_ar_id]["EFF_DATE"])) for fld_ar_id in dups}
flood["INEFFDATE"] = flood.apply(calc_ineffdate, date_dict=dup_dates, axis=1)
flood.rename(columns={"EFF_DATE": "ADOPTDATE"}, inplace=True)

In [ ]:
def calc_floodplain(row):
    if row["SFHA_TF"] == "T":
        if row["ZONE_SUBTY"] == 'FLOODWAY':
            floodplain = 'Conveyance Zone'
        else:
            floodplain = '100-Year'
    else:
        if row["ZONE_SUBTY"] in ('0.2 PCT ANNUAL CHANCE FLOOD HAZARD', 'AREA WITH REDUCED FLOOD RISK DUE TO LEVEE'):
            floodplain = '500-Year'
        else:
            floodplain = None
    return floodplain

def calc_femazone(row):
    if row["FLD_ZONE"] == 'AO':
        zone = 'AO' + str(round(row['DEPTH']))
    elif row["FLD_ZONE"] == 'AH':
        zone = 'AH' + str(round(row["STATIC_BFE"]))
    else:
        zone = row["FLD_ZONE"]
    return zone


flood["FLOODPLAIN"] = flood.apply(calc_floodplain, axis=1)
flood["FEMAZONE"] = flood.apply(calc_femazone, axis=1)
flood.loc[flood["INEFFDATE"].notnull(), "LIFECYCLE"] = "Inactive"
flood.loc[flood["INEFFDATE"].isnull(), "LIFECYCLE"] = "Active"
flood["SOURCE"] = "FEMA"

In [ ]:
city_flood_url = "https://maps.bouldercolorado.gov/arcgis3/rest/services/util/Floodplain/MapServer/3"
city_flood = FeatureLayer(city_flood_url)
sfha_compare = city_flood.query(out_fields=['DRAINAGE'], out_sr=2876)

In [ ]:
# Union the city drainages
def union_drainages(row):
    no_sr = union(geometries=row["SHAPE"], spatial_ref=sr, gis=anon_gis)
    geom = Geometry({"rings": no_sr.rings, "spatialReference": {"wkid": sr}})
    return geom

drainages = pd.DataFrame(sfha_compare.sdf.groupby("DRAINAGE")["SHAPE"].apply(list)).reset_index()
drainages["SHAPE"] = drainages.apply(union_drainages, axis=1)
drain_fs = FeatureSet.from_dataframe(drainages)

In [ ]:
def rand_point_in_poly(row):
    rand = row["SHAPE"].as_shapely.representative_point()
    point = Geometry({"x": rand.x, "y": rand.y, "spatialReference": {"wkid": sr}})
    return point

def calc_drainage(row):
    for item in drain_fs.features:
        arcgis_point = rand_point_in_poly(row)
        g = Geometry(item.geometry)
        if g.contains(arcgis_point):
            drain = item.attributes["DRAINAGE"]
            return drain

# Calc drainages over
flood["DRAINAGE"] = flood.apply(calc_drainage, axis=1)

In [ ]:
# Drop all non-floodplain areas and non-essential columns
flood = flood[flood["ZONE_SUBTY"] != "AREA OF MINIMAL FLOOD HAZARD"]
flood = flood[["FLOODPLAIN", "DRAINAGE", "FEMAZONE", "LIFECYCLE", "ADOPTDATE", "INEFFDATE", "SOURCE", "SHAPE"]]